In [1]:
#  Functions to find all the Julia files in a directory and subdirectories
function filter_jl_files(dir_list::Vector{String})
    """Filter a list of filenames into a list of Julia files and a list of non-Julia files"""
    jl_files = filter(x -> endswith(x, ".jl"), dir_list)
    non_jl_files = setdiff(dir_list, jl_files)
    return jl_files, non_jl_files
end

function add_filepaths!(filepaths::Dict{String, String}, dir_path::String, dir_list::Vector{String})
    """Add a list of filepaths to a dictionary"""
    for file in dir_list
        filepaths[file] = joinpath(dir_path, file)
    end
end

function find_jl_files(dir_path::String)
    """Find all Julia files in a directory and subdirectories"""
    filepaths = Dict{String, String}()
    jl_files, non_jl_files = filter_jl_files(readdir(dir_path))
    add_filepaths!(filepaths, dir_path, jl_files)
    new_dirs = filter(x -> isdir(joinpath(dir_path, x)), non_jl_files)
    for new_dir in new_dirs
        merge!(filepaths, find_jl_files(joinpath(dir_path, new_dir)))
    end
    return filepaths
end

find_jl_files (generic function with 1 method)

In [2]:
# Functions to read a Julia file, clean it, and return it as a long string
function clean_file(dir_path::String)
    """
        filestring = clean_file(dir_path::String)

    Read a Julia file line by line,
    removing all comments and doc strings,
    then appending each line to a string with a space
    """
    open(dir_path, "r") do file
        filestring = ""
        long_comment = false
        for line in eachline(file)
            # Check for doc strings and ignore them
            if contains(line, """\"\"\"""")
                long_comment = !long_comment
            end
            if long_comment
                continue
            else 
                # Remove comments
                if !startswith(strip(line), "#") && line != ""
                    # Remove end-of-line commments
                    line = split(line, "#")[1]
                    # Remove tabs
                    line = replace(line, "\t" => "  ")
                    # Make all one line, but add space to avoid concatenating words
                    filestring = string(filestring, " ", line)
                end
            end
        end
        return filestring
    end
end

function make_filestrings(filepath::Dict{String, String})
    """Make a dictionary of filestrings from a dictionary of filepaths"""
    filestrings = Dict{String, String}()
    for (filename, filepath) in filepaths
        filestrings[filename] = clean_file(filepath)
    end
    return filestrings
end

make_filestrings (generic function with 1 method)

In [3]:
# Functions to find the names and creation-file of model features from a filestring
function extract_feature_name(filestring::AbstractString, filter_prefix::AbstractString)
    """Extract the name of a model feature from a file-substring"""
    x = split(filestring, ",")
    # Try to extract the name based on a prefix
    x2 = split(x[2], " ")
    x2 = filter(x -> startswith(x, filter_prefix), x2)
    if !isempty(x2)
        x = x2[1]
    else
        x = x[2]
    end
    # Remove any indexing
    if contains(x, "[")
        x = split(x, "[")[1]
    end
    # Remove whitespace
    return strip(x)
end

function find_feature_names(filestring::String, filter_string::String)
    """Create a list of the names of all the model feature of a given type, from a filestring"""
    cases = split(filestring, filter_string)
    feature_names = Vector{String}()
    filter_prefix = string(filter_string[2])
    for i in 2:length(cases)
        push!(feature_names, extract_feature_name(cases[i], filter_prefix))
    end
    return feature_names
end

function find_feature_creation!(feature_results::Dict{String, Any}, filestrings::Dict{String, String}, feature_name::String)
    """Find the files in which a set of features is created"""
    for (filename, filestring) in filestrings
        new_feature_names = find_feature_names(filestring, feature_name)
        for new_name in new_feature_names
            if !haskey(feature_results, new_name)
                feature_results[new_name] = Dict{String, Any}()
            end
            feature_results[new_name]["created"] = filename
        end
    end
end

find_feature_creation! (generic function with 1 method)

In [4]:
# Functions to find where each model feature is accessed
function check_access(filestrings::Dict{String, String}, feature_names::Vector{String})
    """Check if features are accessed in a set of files, using filestrings"""
    file_accesses = Dict{String, Vector{String}}()
    for (filename, filestring) in filestrings
        file_accesses[filename] = Vector{String}()
        for feature_name in feature_names
            if contains(filestring, feature_name)
                push!(file_accesses[filename], feature_name)
            end
        end
    end
    return file_accesses
end

function push_accesses!(feature_results::Dict{String, Any}, accesses::Dict{String, Vector{String}})
    """Push a list of features accesses to a feature result dictionary"""
    for (filename, feature_names) in accesses
        for feature_name in feature_names
            if !haskey(feature_results[feature_name], "accessed")
                feature_results[feature_name]["accessed"] = Vector{String}()
            end
            push!(feature_results[feature_name]["accessed"], filename)
        end
    end
end

function find_feature_accesses!(feature_results::Dict{String, Any}, filestrings::Dict{String, String})
    """Find the files in which a set of features is accessed"""
    accesses = check_access(filestrings, collect(keys(feature_results)))
    push_accesses!(feature_results, accesses)
end

find_feature_accesses! (generic function with 1 method)

In [5]:
# Functions to turn search_results into a Markdown table
function make_table_header(headers::Set{String}, feature_type::String)
    fun_headers = copy(headers)
    table_header = "|$(feature_type) name"
    table_divider = "|:-"
    if "created" in fun_headers # Later we can automate this using the features_to_search superdetails
        table_header = string(table_header, "|created")
        table_divider = string(table_divider, "|:-")
        delete!(fun_headers, "created")
    end    
    for header in fun_headers
        table_header = string(table_header, "|", header)
        table_divider = string(table_divider, "|", ":-")
    end
    table_header = string(table_header, "|")
    table_divider = string(table_divider, "|")
    return table_header, table_divider
end

function make_table_row(feature_name::String, feature_results::Dict{String, Any}, headers::Set{String})
    table_row = "|$(feature_name)"
    fun_headers = copy(headers)
    if "created" in fun_headers
        table_row = string(table_row, "|", feature_results["created"])
        delete!(fun_headers, "created")
    end
    for header in fun_headers
        table_row = string(table_row, "|", join(feature_results[header], ", "))
    end
    table_row = string(table_row, "|")
    return table_row
end

function features_2_markdown(feature_results::Dict{String, Any}, feature_type::String, headers::Set{String})
    md_table = Vector{String}()
    push!(md_table, "## $(feature_type)s")
    table_header, table_divider = make_table_header(headers, feature_type)
    push!(md_table, table_header)
    push!(md_table, table_divider)
    sorted_names = sort(collect(keys(feature_results)))
    for name in sorted_names
        push!(md_table, make_table_row(name, feature_results[name], headers))
    end
    return md_table
end

function write_2_file(filename::String, md_table::Vector{String})
    open(filename, "w") do f
        for line in md_table
            write(f, string(line, "\n"))
        end
    end
end

function write_2_file(filename::String, md_table::Vector{String}, open_mode::String)
    open(filename, open_mode) do f
        for line in md_table
            write(f, string(line, "\n"))
        end
    end
end

write_2_file (generic function with 2 methods)

In [6]:
# This finds all the variables and expressions, and prints them as a markdown table

# Get all Julia files in the src directory
src_path = joinpath(dirname(pwd()), "src")
filepaths = find_jl_files(src_path)

outpath = joinpath(pwd(), "model_table.md")
open(outpath, "w") do io
    println(io, "# Model Variables and Expressions")
end

# Clean all Julia files and return them as strings
filestrings = make_filestrings(filepaths)

# Create a dictionary of all the model features we'd like to search for, and the details of interest
features_to_search = Dict(
    "var" => Dict("filter_string" => "@variable", "table_name" => "Variable", "details" => ["created", "accessed"], "superdetails" => ["created"]),
    "exp" => Dict("filter_string" => "@expression", "table_name" => "Expression", "details" => ["created", "accessed"], "superdetails" => ["created"])
)

# Create dictionary to hold search results
# We could be more specific and make the last Dict{String, String||List}, but this is safer
search_results = Dict{String, Dict{String, Any}}()
for (search_name, search_details) in features_to_search
    search_results[search_name] = Dict{String, Dict{String, Any}}()
end

# Our dict of results is not ordered, so we specify that here
print_order = Vector{String}(["var", "exp"])

# Find the file in which each feature is created
for search_name in print_order
    search_details = features_to_search[search_name]
    find_feature_creation!(search_results[search_name], filestrings, search_details["filter_string"])
    find_feature_accesses!(search_results[search_name], filestrings)
    md_table = features_2_markdown(search_results[search_name], search_details["table_name"], Set(search_details["details"]))
    write_2_file(outpath, md_table, "a")
end

In [7]:
# Functions to find all functions in a file and their arguments
function extract_function_names_and_args(filestring::String)
    """Extract function names and arguments from a filestring"""
    function_args = Dict{String, Vector{String}}()
    filesubstrings = split(filestring, "function ")
    filesubstrings = filesubstrings[2:end]
    for substring in filesubstrings
        substring = split(substring, "(")
        function_name = substring[1]
        function_arg = split(split(substring[2], ")")[1], ", ")
        function_args[function_name] = function_arg
    end
    return function_args
end

function find_functions(filestrings::Dict{String, String})
    """Find all functions in a set of filestrings"""
    functions_dict = Dict{String, Dict{String, Vector{String}}}()
    for (filename, filestring) in filestrings
        functions_dict[filename] = extract_function_names_and_args(filestring)
    end
    return functions_dict
end

functions_dict = find_functions(filestrings)

Dict{String, Dict{String, Vector{String}}} with 145 entries:
  "h2_production_commit.jl"          => Dict("h2_production_commit"=>["EP::Mode…
  "write_reserve_margin_revenue.jl"  => Dict("write_reserve_margin_revenue"=>["…
  "load_co2_cap.jl"                  => Dict("load_co2_cap"=>["setup::Dict", "p…
  "flexible_demand.jl"               => Dict("flexible_demand"=>["EP::Model", "…
  "investment_discharge.jl"          => Dict("investment_discharge"=>["EP::Mode…
  "write_reg.jl"                     => Dict("write_reg"=>["path::AbstractStrin…
  "write_time_weights.jl"            => Dict("write_time_weights"=>["path::Abst…
  "load_inputs.jl"                   => Dict("load_inputs"=>["setup::Dict,path:…
  "emissions_power.jl"               => Dict("emissions_power"=>["EP::Model", "…
  "load_period_map.jl"               => Dict("load_period_map"=>["setup::Dict,p…
  "h2_pipeline.jl"                   => Dict("h2_pipeline"=>["EP::Model", "inpu…
  "load_h2_g2p_variability.jl"       => Dict("lo

In [8]:
function find_expression_mutations(filestring::String, expression_name::String, model_name::String)
    """
    Find all mutations of an expression in a file, based on a filestring\n
    TODO: This is a really weak approach and would be better to handle with the parser.
    """
    expression_mutations = Vector{String}()
    # Assume that these expression mutations are not indexed, so we add a space after the name
    exp_filter_string = string(model_name, "[:", expression_name, "] ")
    filesubstrings = split(filestring, exp_filter_string)[2:end]
    for substring in filesubstrings
        # For now assume that all mutations are single lines -- TODO: This is really weak
        mut = split(substring, "  ")[1]
        # Simple check that what we've found is a mutation
        try Meta.parse(string(exp_filter_string, " ", mut))
            push!(expression_mutations, mut)
        catch
            continue
        end
    end
    return expression_mutations
end

function exp_2_markdown(exp_mutations::Dict{String, Vector{String}}, exp_name::String)
    md_table = Vector{String}()
    push!(md_table, "## $(exp_name)")
    table_header = "|Mutation|File|Explanation| "
    table_divider = "|:-|:-|:-"
    push!(md_table, table_header)
    push!(md_table, table_divider)
    sorted_names = sort(collect(keys(exp_mutations)))
    for name in sorted_names
        for mut in exp_mutations[name]
            push!(md_table, string("|", mut, "|", name, "|", " ", "|"))
        end
    end
    return md_table
end

exp_2_markdown (generic function with 1 method)

In [9]:
# Options looking at the Julia parser
s = "string(\"Ruaridh\", \" Jamie\")"
s = "1 +"
w = Meta.parse(s)
typeof(w)
println(w.head)
println(w.args)

incomplete


Any["incomplete: premature end of input"]


In [10]:
outpath = joinpath(pwd(), "expression_mutations_table.md")
open(outpath, "w") do io
    println(io, "# Expression Mutations")
end

exp_names = [
    "eObj", 
    "ePowerBalance", 
    "eH2Balance", 
    ] 

expression_mutations = Dict{String, Dict{String, Vector{String}}}()
for exp_name in exp_names
    expression_mutations[exp_name] = Dict{String, Vector{String}}(exp_name => Vector{String}())
    for (filename, filestring) in filestrings
        mutations = find_expression_mutations(filestring, exp_name, "EP")
        if length(mutations) > 0
            expression_mutations[exp_name][filename] = mutations
        end
    end
    md_table = exp_2_markdown(expression_mutations[exp_name], exp_name)
    write_2_file(outpath, md_table, "a")
end

expression_mutations

Dict{String, Dict{String, Vector{String}}} with 3 entries:
  "eObj"          => Dict("h2_production_commit.jl"=>["+= eTotalH2GenCStart"], …
  "ePowerBalance" => Dict("h2_production_commit.jl"=>["+= -ePowerBalanceH2GenCo…
  "eH2Balance"    => Dict("h2_pipeline.jl"=>["+= ePipeZoneDemand"], "h2_product…

In [11]:
# Method to find @constraints in a filestring
# This is much more challenging as they're not named
# And many of them are only made under certain conditions
# And can be mutually exclusive.
# Naming the constraints may avoid accidental conflicting constraints
filestring = filestrings["thermal_no_commit.jl"]
filesubstrings = split(filestring, "@constraints(")
substring = filesubstrings[2]
substring = split(substring, "begin")[2]
substring = split(substring, "end")[1]
substring = split(substring, "   ")[2]
Meta.parse(substring).args
# for s in substring
    # delete!()
# end
# substring = split(substring, r"[<=>]=")
# substring = split("substring", ["==", ">=", "<="])
# Meta.parse(substring)


2-element Vector{Any}:
 :([y in THERM_NO_COMMIT, t in START_SUBPERIODS])
 :((EP[:vP])[y, t] - (EP[:vP])[y, (t + hours_per_subperiod) - 1] <= (dfGen[!, :Ramp_Up_Percentage])[y] * (EP[:eTotalCap])[y])

In [12]:
open(dir_path, "r") do file
    filestring = ""
    long_comment = false
    for line in eachline(file)
        # Check for doc strings and ignore them
        if contains(line, """\"\"\"""")
            long_comment = !long_comment
        end
        if long_comment
            continue
        else 
            # Remove comments
            if !startswith(strip(line), "#") && line != ""
                # Remove end-of-line commments
                line = split(line, "#")[1]
                # Remove tabs
                line = replace(line, "\t" => "  ")
                # Make all one line, but add space to avoid concatenating words
                filestring = string(filestring, " ", line)
            end
        end
    end
    return filestring
end

UndefVarError: UndefVarError: dir_path not defined

In [31]:
function minify_file(filepath::String)
    """Minify a file"""
    minified_lines = Vector{String}()
    open(filepath, "r") do io
        long_comment = false
        for line in eachline(io)
            # Check for doc strings and ignore them
            if contains(line, """\"\"\"""")
                long_comment = !long_comment
                continue
            end
            if !long_comment && !startswith(strip(line), "#") && line != ""
                    # Remove end-of-line commments
                    line = split(line, "#")[1]
                    # Remove tabs
                    line = replace(line, "\t" => "")
                    # Push to output vector
                    push!(minified_lines, line)
            end
        end
        return minified_lines
    end
end

function parse_lines(lines::Vector{String})
    """Parse a vector of lines"""
    output_exprs = Vector{Any}()
    active_str = ""
    for line in lines
        active_str = string(active_str, "\n", line)
        try Meta.parse(active_str)
            parse = Meta.parse(active_str)
            if parse.head == :incomplete
                continue
            end
            reduced_lines = [split(active_str, "\n")][2:end-1]
            println(reduced_lines)
            push!(output_exprs, parse_lines(reduced_lines))
            active_str = ""
        catch
            continue
        end
    end
    return output_exprs
end

filename = "thermal_no_commit.jl"
filepath = filepaths[filename]
lines = minify_file(filepath)
file_exprs = parse_lines(lines)


Vector{SubString{String}}[]


Any[]

In [33]:
parse_lines(Vector{String}())

Any[]

In [23]:
filename = "thermal_no_commit.jl"
filepath = filepaths[filename]
output_lines = Vector{String}()
out2 = Vector{String}()
open(filepath, "r") do io
    active_str = ""
    long_comment = false
    for line in eachline(io)
        # Check for doc strings and ignore them
        if contains(line, """\"\"\"""")
            long_comment = !long_comment
            continue
        end
        if !long_comment
            # Remove comments
            if !startswith(strip(line), "#") && line != ""
                # Remove end-of-line commments
                line = split(line, "#")[1]
                # Remove tabs
                line = replace(line, "\t" => "")
                active_str = string(active_str, "\n", line)
                push!(out2,active_str)
                try Meta.parse(active_str)
                    parse = Meta.parse(active_str)
                    if parse.head == :incomplete
                        continue
                    end
                    # if parse.head == :incomplete && !startswith(strip(active_str), "function")
                        # continue
                    # end
                    push!(output_lines, active_str)
                    active_str = ""
                catch
                    continue
                end
            end
        end
    end
end
output_lines

2-element Vector{String}:
 "\nfunction thermal_no_commit(EP" ⋯ 1412 bytes ⋯ "Cap][y]\nend)\nend\nreturn EP\nend"
 "\nfunction thermal_no_commit_re" ⋯ 2206 bytes ⋯ "Cap][y]\nend)\nend\nreturn EP\nend"